<h1><center>Data Analysis</center></h1>
<h2><center>Kaggle report notebook</center></h2>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,8)

In [2]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV, KFold

# Introduce yourself

Put your team name and team-members' names below

Zhanibek Taubaldiyev<br />
Alexander Tsay<br />
Denis Kumar<br />
Dmitriy Dolgosheyev

# Task

In this task we are working with posts on famous russian advertisement and resell platform.

Given the information about the post and corresponding item you have to estimate its probability to be sold in 3 days.

Below you will find some desirable stages of your report.

# Data Loading

In [3]:
df_train = pd.read_csv('train_data.tsv.gz', sep='\t', index_col=0)
df_test = pd.read_csv('test_data.tsv.gz', sep='\t', index_col=0)

In [4]:
df_train

,city,price,brand,model,drive_type,engine_summary,owner_type,generation,year,color,body,gear_type,wheel_type,state,is_new,doors_count,mileage,owners_by_pts
id,,,,,,,,,,,,,,,,,,
9cbd65bfd61bc7405d6866a5,Советск,40000.0,ВАЗ (Lada),2105,Задний,1.5 л / 71 л.с. / Бензиновый,Частники,1 поколение,1999,Фиолетовый,Седан,Механика,Левый,Хорошее,Нет,5.0,178000.0,3.0
5d6fa4d552284029123966a5,Москва,105000.0,ВАЗ (Lada),2114,Передний,1.5 л / 77 л.с. / Бензиновый,Частники,1 поколение,2006,Голубой,Хетчбэк,Механика,Левый,Хорошее,Нет,5.0,200000.0,2.0
52fd95e853654e3c4d7966a5,Александров,650000.0,Cadillac,SRX,Полный,3.6 л / 258 л.с. / Бензиновый,Частники,1 поколение,2008,Синий,Кроссовер,Автомат,Левый,Не битый,Нет,5.0,185000.0,3.0
beb7c749571bc740820038a5,Новокуйбышевск,400000.0,Chevrolet,Lacetti,Передний,1.6 л / 109 л.с. / Бензиновый,Частники,1 поколение,2013,Красный,Седан,Механика,Левый,Не битый,Нет,4.0,59000.0,1.0
34d7824a60964435b93c68a5,Москва,1000000.0,Jeep,Commander,Полный,3 л / 218 л.с. / Дизельный,Частники,1 поколение,2008,Черный,Внедорожник,Автомат,Левый,Не битый,Нет,5.0,200000.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3bebe572d5ab5606c5846fd5,Балашиха,327000.0,ВАЗ (Lada),Granta,Передний,1.6 л / 87 л.с. / Бензиновый,Частники,1 поколение,2015,Черный,Лифтбэк,Механика,Левый,Не битый,Нет,5.0,26000.0,1.0
311871e2a71bc740dc356fd5,Дагестанские Огни,35000.0,ВАЗ (Lada),2110,Передний,1.5 л / 71 л.с. / Бензиновый,Частники,1 поколение,2000,Белый,Седан,Механика,Левый,Среднее,Нет,4.0,250628.0,5.0
37e2562ad78bf5750c586fd5,Энгельс,35000.0,ВАЗ (Lada),2107,Задний,1.6 л / 73 л.с. / Бензиновый,Частники,1 поколение,2005,Белый,Седан,Механика,Левый,Хорошее,Нет,4.0,90000.0,4.0


# Descriptive Data Analysis

* Analyze train dataset features
    * Provide some meaningful description about corresponding features (in form of pictures or some coefficients)
    * Outline any findings you are able to reveal
* Try to measure analyse any correspondense between features and target features

After an extensive evaluation of available features in the data set we have come to a decision to divide engine_summary into three separate columns of features. We also dropped 'id' column as it does not have any effect on predicted results. 'Price' column is our label vector, so we drop it from X_train as well. 

In [5]:
df_train['litres'], df_train['#hrsprs'], df_train['benzo'] = df_train['engine_summary'].str.split('/', 2).str
df_train = df_train.drop(['engine_summary'], axis=1)

C:\Users\tzhan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  """Entry point for launching an IPython kernel.


In [6]:
df_train['litres'] = df_train['litres'].str.replace(' л', '')
df_train['#hrsprs'] = df_train['#hrsprs'].str.replace(' л.с.', '')

In [7]:
df_train[df_train.columns[0]]

id
9cbd65bfd61bc7405d6866a5              Советск
5d6fa4d552284029123966a5               Москва
52fd95e853654e3c4d7966a5          Александров
beb7c749571bc740820038a5       Новокуйбышевск
34d7824a60964435b93c68a5               Москва
                                  ...        
3bebe572d5ab5606c5846fd5             Балашиха
311871e2a71bc740dc356fd5    Дагестанские Огни
37e2562ad78bf5750c586fd5              Энгельс
3ca325dd8af2e08aa0b47fd5               Москва
3723613653654e3c36667fd5               Казань
Name: city, Length: 89959, dtype: object

# Machine-learning pipeline and prediction

Here you develop your pipeline, generate features for train and test sets
* Your pipeline should be reproducible - make sure you set all `random_states` if needed

We tried different scalers in pipeline. The best result was when we used MinMaxScaler. We tried to seperate engine_summary in 3 columns and also we tried to drop engine_summary. When we dropped engine_summary we had a score of approximately 26.5 (using StandardScaler). When we divided endgine_summary it gave us a score of 26.0 with StandardScaler and 25.6 with MinMaxScaler. We also tried CatBoostRegressor with default params and got a score of 34.

In [8]:
y_train = df_train.loc[:, 'price'].values

In [9]:
df_train = df_train.drop(['price'], axis=1)

In [10]:
X_train = df_train.values

In [11]:
df_train.columns

Index(['city', 'brand', 'model', 'drive_type', 'owner_type', 'generation',
       'year', 'color', 'body', 'gear_type', 'wheel_type', 'state', 'is_new',
       'doors_count', 'mileage', 'owners_by_pts', 'litres', '#hrsprs',
       'benzo'],
      dtype='object')

In [12]:
categ = [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 18] # categorical feauture index
categ_preprocess = \
Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore'))    
])
# firstly - impute missing values
# One hot encodding afterwards

numeric = [6, 14, 15, 16, 17] # numeric feauture index
numeric_preprocess = \
Pipeline([
    ('impute', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])
# firstly - impute missing values
# standart scaling afterwards

col_transformer = ColumnTransformer([
    ('categorical', categ_preprocess, categ),
    ('numeric', numeric_preprocess, numeric),
])
# combine transformations

In [13]:
model = \
Pipeline([
    ('preprocess', col_transformer),
    ('model', Ridge())
])
# pipeline with preprocessing and models

In [14]:
param_grid = \
{'model__alpha': np.logspace(-3, 3, 100),
'preprocess__numeric__impute__strategy': ['mean', 'median']}

# setting cv scheme
cv = KFold(n_splits=5, shuffle=True, random_state=123)

# setting hyperparameter search
searcher = \
RandomizedSearchCV(model, param_grid, n_iter=20, cv=cv, 
                   scoring='neg_mean_squared_error',
                   n_jobs=5, refit=True, verbose=2, 
                   random_state=123)

In [15]:
searcher.fit(X_train, np.log(y_train+1))

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed:  8.2min
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed: 23.7min finished


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=123, shuffle=True),
                   estimator=Pipeline(steps=[('preprocess',
                                              ColumnTransformer(transformers=[('categorical',
                                                                               Pipeline(steps=[('impute',
                                                                                                SimpleImputer(strategy='most_frequent')),
                                                                                               ('ohe',
                                                                                                OneHotEncoder(handle_unknown='ignore'))]),
                                                                               [0,
                                                                                1,
                                                                                2,
                                      

In [16]:
best_model = searcher.best_estimator_

In [17]:
df_test = pd.read_csv('test_data.tsv.gz', sep='\t', index_col=0)

In [18]:
df_test['litres'], df_test['#hrsprs'], df_test['benzo'] = df_test['engine_summary'].str.split('/', 2).str
df_test = df_test.drop(['engine_summary'], axis=1)
df_test['litres'] = df_test['litres'].str.replace(' л', '')
df_test['#hrsprs'] = df_test['#hrsprs'].str.replace(' л.с.', '')

C:\Users\tzhan\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  """Entry point for launching an IPython kernel.


In [19]:
X_test = df_test.values

In [20]:
y_hat = \
np.exp(best_model.predict(X_test))-1

In [21]:
submission = df_test.drop(list(df_test.columns.drop(['price'])), axis=1)
submission.loc[:, 'price'] = y_hat

KeyError: "['price'] not found in axis"

In [ ]:
submission['id'] = df_test.index

In [ ]:
submission

In [ ]:
columns_titles = ["id","price"]
submission=submission.reindex(columns=columns_titles)

In [ ]:
submission.to_csv('baseline.csv', index=None)

# Model interpretation
Try to interpret your model results
* What **knowledge** does it reveal?